In [1]:
# pip install scrapy
# pip install --upgrade zope2

from collections import Counter
import foursquare
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import random
import requests
from scrapy import Selector
from scrapy.http import HtmlResponse
import seaborn as sns
from sqlalchemy import create_engine
import sqlite3
import textacy
import textacy.data
import time
import unicodedata

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
usda = pd.read_pickle('../../../../Accounts/Electo:Computad/\
MacBook Pro/Software/Github/Old_repos/DSI-SF-4-anotherguitar/projects/Capstone Stuff/usda.pkl')
food = pd.read_pickle('../../../../Accounts/Electo:Computad/\
MacBook Pro/Software/Github/Old_repos/DSI-SF-4-anotherguitar/projects/Capstone Stuff/explore_endpoint_df.pkl')
vegan = food[food.vegan_venue_check == 'vegan']

In [3]:
def word_counter(unique_desc):
    single_string = ' '.join(unique_desc)
    count_string = len(single_string.split())
    return count_string

In [4]:
# have to count the number of words in each list and add this as a new column to the dataframe
update_to_food = food
update_to_food['count_desc_list'] = food.venue_desc_list.map(lambda x: word_counter(x))

In [5]:
def item_desc_counter(phrase):
    if phrase == 'desc_not_available':
        return 0
    else:
        countered = len(phrase.split())
        return countered

In [6]:
update_to_food['count_item_name'] = food.menu_item_name.map(lambda x: len(x.split()))
update_to_food['count_item_desc'] = food.menu_item_desc.map(lambda x: item_desc_counter(x))

In [7]:
update_to_food.head()
vegan_update = update_to_food[update_to_food.vegan_venue_check == 'vegan']
vegan_update.head(2)

,venue_name,venue_desc_list,vegan_venue_check,venue_menu_url,venue_rated,meta_menu_n,depth_menus_n,menu_item_name,menu_item_price,menu_item_desc,count_desc_list,count_item_name,count_item_desc
1064,Amber India,"[Indian Restaurant, Vegetarian / Vegan Restaur...",vegan,https://foursquare.com/v/amber-india/4acab395f...,8.6,Dessert Menu,"Ports, Madeira, & Dessert Wines","Tokaji, Disznoko, Kesoi Szuret, Hungary '08",9,desc_not_available,6,6,0
1065,Amber India,"[Indian Restaurant, Vegetarian / Vegan Restaur...",vegan,https://foursquare.com/v/amber-india/4acab395f...,8.6,Dessert Menu,"Ports, Madeira, & Dessert Wines","Tawny Port, Graham's 20 Yr",11,desc_not_available,6,5,0


In [8]:
usda = usda.drop('ComName', axis=1)
usda = usda.drop('FG_Cd', axis=1)

In [9]:
usda.head()

,Food_No,Food_Group,Description,Nutr_No,Nutrient,Amount_per_100,Units,Tagname,Not_Imputed
0,01001,Dairy and Egg Products,"Butter, salted",203,Protein,0.85,g,PROCNT,1
1,01001,Dairy and Egg Products,"Butter, salted",204,Total lipid (fat),81.11,g,FAT,1
2,01001,Dairy and Egg Products,"Butter, salted",205,"Carbohydrate, by difference",0.06,g,CHOCDF,0
3,01001,Dairy and Egg Products,"Butter, salted",207,Ash,2.11,g,ASH,1
4,01001,Dairy and Egg Products,"Butter, salted",208,Energy,717.00,kcal,ENERC_KCAL,0


In [10]:
dads = usda[['Food_Group','Description']]
dads.head()

,Food_Group,Description
0,Dairy and Egg Products,"Butter, salted"
1,Dairy and Egg Products,"Butter, salted"
2,Dairy and Egg Products,"Butter, salted"
3,Dairy and Egg Products,"Butter, salted"
4,Dairy and Egg Products,"Butter, salted"


In [11]:
# Let's make a dictionary with the 25 unique food groups and their respective descriptions
deduped = []
for group, group_df in dads.groupby(['Food_Group', 'Description']):
    deduped.append({
      "Food_Group":  group[0],
      "Description": group[1]
    })

In [12]:
# Example
print deduped[2]['Food_Group'].lower().replace(',',' ').replace('/',' ').replace('-',' ').replace('_',' ').replace('.',' ').replace(',',' ') 
print deduped[2]['Description'].lower().replace(',',' ').replace('/',' ').replace('-',' ').replace('_',' ').replace('.',' ').replace(',',' ') 
print len(deduped)

american indian alaska native foods
agave  dried (southwest)
8194


In [16]:
# This function will assign food groups given a single word input

def assign_group(word_in_item):
    groups = []
    for item_no in range(len(deduped)):
        search_this = deduped[item_no]['Description'].lower().replace(',','').replace('/',' ').replace('-',' ').replace('_',' ').replace('.',' ').replace('(','').replace(')','').replace("'","").replace('&','')
        if search_this.find(word_in_item.lower()) != -1:
            groups.append(deduped[item_no]['Food_Group'])
            groupings = list(set(groups))
            groupings = ', '.join(groupings)
            return groupings

In [17]:
# This function will assign food groups if given a feature from a particular item as an input

def groups_from_item(itemed):
    
    #clean item
    item = itemed.lower().replace(',','').replace('/',' ').replace('-',' ').replace('_',' ').replace('.',' ').replace('(','').replace(')','').replace("'","").replace('&','')
    
    # create a placeholder (or a list) with all the applicable food groups for a given food item    
    all_food_groups = 'food_groups_placeholder'

    if len(item.split()) == 1:
    # because there's just one word, we can feed the item directly into the search loop
        word_in_item = item
        all_food_groups = assign_group(word_in_item)
        return all_food_groups
        
    # this is how I'll map the series with the correct menu item on the larger dataframe,
    # just in case I decide not to use the series map function directly

    elif len(item.split()) >= 1:
        # because there's more than one word, I'll have to do an extra loop to hit each word, but making sure that
        # they get appended to the same row, and not to the following row
        each_words_group = []
        for word_in_item in item.split():
            a_food_group = assign_group(word_in_item)
            if a_food_group != None:
                each_words_group.append(a_food_group)
        if each_words_group != []:
            all_food_groups = ', '.join(each_words_group)
        else:
            all_food_groups = 'no_groups_avail'
        return all_food_groups
        
# I could just create a list of possible food_groups first, then determine which food group is more likely? 
# for example, by taking the most common food group chosen for each of the words in that particular menu item

# ^^ I think the best way to do this is to make fg columns for item, menu, sub_menu, etc. and then pick the
# group label with the highest label, or maybe the top 2/3 labels per row

In [18]:
vegan_next_version = vegan
list_run_thru = vegan_next_version.menu_item_name
print len(list_run_thru)

1861


In [20]:
# Here I've automated the food group tagging I would have done across each item, sub menu, and meta menu
start_time = time.time()
vegan_next_version['item_fg'] = vegan_next_version.menu_item_name.map(lambda x: groups_from_item(x))
vegan_next_version['depth_menu_fg'] = vegan_next_version.depth_menus_n.map(lambda x: groups_from_item(x))
vegan_next_version['meta_menu_fg'] = vegan_next_version.meta_menu_n.map(lambda x: groups_from_item(x))
print("--- %s seconds ---" % (time.time() - start_time))

/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [22]:
vegan_next_version[['venue_name','venue_rated','menu_item_name','menu_item_price','item_fg','depth_menu_fg','meta_menu_fg']].head(1)

,venue_name,venue_rated,menu_item_name,menu_item_price,item_fg,depth_menu_fg,meta_menu_fg
1064,Amber India,8.6,"Tokaji, Disznoko, Kesoi Szuret, Hungary '08",9,no_groups_avail,"Beverages, Baby Foods","Baby Foods, Restaurant Foods"


In [23]:
vegan_next_version[vegan_next_version.menu_item_name == 'Lemon Grass Panna Cotta']

,venue_name,venue_desc_list,vegan_venue_check,venue_menu_url,venue_rated,meta_menu_n,depth_menus_n,menu_item_name,menu_item_price,menu_item_desc,item_fg,depth_menu_fg,meta_menu_fg
1082,Amber India,"[Indian Restaurant, Vegetarian / Vegan Restaur...",vegan,https://foursquare.com/v/amber-india/4acab395f...,8.6,Dessert Menu,Dessert Menu,Lemon Grass Panna Cotta,8,Lemon Grass & Kafir Lime Leaves set Custard,"Baked Products, Baked Products, Dairy and Egg ...","Baby Foods, Restaurant Foods","Baby Foods, Restaurant Foods"


In [24]:
# Below is extra data that I might later use to correlate nutrients with particular food groups,
# though I'm not sure if there will be enough time for me to complete this aspect of the project

In [25]:
usda.shape

(595359, 9)

In [26]:
tmp = usda[['Food_Group','Nutrient']]

In [27]:
tmp = tmp.groupby(['Food_Group','Nutrient']).agg(lambda x: 1.)
tmp.head()

Food_Group                           Nutrient
American Indian/Alaska Native Foods  10:0        1.0
                                     12:0        1.0
                                     14:0        1.0
                                     14:1        1.0
                                     15:0        1.0
dtype: float64

In [28]:
tmp = tmp.unstack(fill_value=0)

In [29]:
tmp.shape

(25, 145)

In [30]:
tmp = tmp.reset_index()

In [31]:
tmp['Carbohydrate, by difference']

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
Name: Carbohydrate, by difference, dtype: float64

In [32]:
tmp.columns.unique()

array(['Food_Group', '10:0', '12:0', '13:0', '14:0', '14:1', '15:0',
       '15:1', '16:0', '16:1 c', '16:1 t', '16:1 undifferentiated', '17:0',
       '17:1', '18:0', '18:1 c', '18:1 t', '18:1 undifferentiated',
       '18:1-11t (18:1t n-7)', '18:2 CLAs', '18:2 i', '18:2 n-6 c,c',
       '18:2 t not further defined', '18:2 t,t', '18:2 undifferentiated',
       '18:3 n-3 c,c,c (ALA)', '18:3 n-6 c,c,c', '18:3 undifferentiated',
       '18:3i', '18:4', '20:0', '20:1', '20:2 n-6 c,c', '20:3 n-3',
       '20:3 n-6', '20:3 undifferentiated', '20:4 n-6',
       '20:4 undifferentiated', '20:5 n-3 (EPA)', '21:5', '22:0', '22:1 c',
       '22:1 t', '22:1 undifferentiated', '22:4', '22:5 n-3 (DPA)',
       '22:6 n-3 (DHA)', '24:0', '24:1 c', '4:0', '6:0', '8:0',
       'Adjusted Protein', 'Alanine', 'Alcohol, ethyl', 'Arginine', 'Ash',
       'Aspartic acid', 'Beta-sitosterol', 'Betaine', 'Caffeine',
       'Calcium, Ca', 'Campesterol', 'Carbohydrate, by difference',
       'Carotene, alpha', 'C

In [33]:
tmp['Sucrose'].head(3).T

0    1.0
1    1.0
2    1.0
Name: Sucrose, dtype: float64

In [34]:
tmp.head(10)

Nutrient,Food_Group,10:0,12:0,13:0,14:0,14:1,15:0,15:1,16:0,16:1 c,...,"Vitamin C, total ascorbic acid",Vitamin D,Vitamin D (D2 + D3),Vitamin D2 (ergocalciferol),Vitamin D3 (cholecalciferol),Vitamin E (alpha-tocopherol),"Vitamin E, added",Vitamin K (phylloquinone),Water,"Zinc, Zn"
0,American Indian/Alaska Native Foods,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1,Baby Foods,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,Baked Products,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Beef Products,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,Beverages,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
5,Breakfast Cereals,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,Cereal Grains and Pasta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
7,Dairy and Egg Products,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,Fast Foods,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
9,Fats and Oils,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
